Importamos librerias

In [28]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.compose import make_column_selector
from sklearn.model_selection import cross_val_score
from sklearn import metrics

Importamos One Hot Encoder

In [29]:
# Archivo utilizado para crear la función de One Hot Encoding
############################################################# 

# Función auxiliar: Toma un df de variables categoricas y selección el top_x de categorias
def one_hot_top_x(cat_df, variable,top_x_labels):
    for label in top_x_labels:
        cat_df[str(variable)+'_'+str(label)] = np.where(cat_df[variable]==label,1,0)

# Función One Hot Encoder: Toma un DF y le hace una transformación de One Hot Encoder para con las categorias más repetidas 
def one_hot_encoder(df,top_x):
    # Separo variables categoricas de númericas
    num_cols = df.select_dtypes(include=['float64', 'int']).columns.to_list()
    cat_cols = df.select_dtypes(include=['object', 'category']).columns.to_list()
    
    # Armo un DF solo con variables categoricas
    cat_df = df.drop(num_cols,axis=1)
    # Armo un DF solo con variables categoricas
    num_df = df.drop(cat_cols,axis=1)
   
    # Itero entre las variables categoricas y me quedo con el top 10 variables
    for variable in cat_cols:
        top_10 = [x for x in cat_df[variable].value_counts().sort_values(ascending=False).head(top_x).index]
        one_hot_top_x(cat_df,variable,top_10)
    
    cat_df = cat_df.drop(cat_cols,axis=1)
    df = pd.concat([cat_df,num_df], axis =1)
    
    return df

Importamos datos

In [86]:
# Levantamos los datos procesados
datos_meli = pd.read_excel('data_ready_to_model.xlsx')
datos_meli['type'] = 'meli' 

# Uno los DF

frames = [datos_meli, datos_privadis]
datos = pd.concat(frames)

datos = one_hot_encoder(datos,100)
datos_meli = datos[(datos['type_meli'] == 1)] 
datos_meli = datos_meli.drop('type_meli',axis=1)

C:\Users\LUCIA~1.LOP\AppData\Local\Temp/ipykernel_16944/187505526.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  cat_df[str(variable)+'_'+str(label)] = np.where(cat_df[variable]==label,1,0)


MODELO REGRESION LINEAL - Base

In [88]:
x = datos_meli.drop('precio',axis=1)

#separte the predicting attribute into Y for model training 
y = datos_meli['precio']

X_train, X_test, y_train, y_test = train_test_split(
                                        x,
                                        y,
                                        train_size   = 0.8,
                                        random_state = 1234,
                                        shuffle      = True
                                    )


# Entrenamos el modelo
model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

# MIramos los resultados
print('Mean Absolute Error:', round(metrics.mean_absolute_error(y_test, y_pred)/1000,2))
print('Mean Squared Error:', round(metrics.mean_squared_error(y_test, y_pred)/1000,2))
print('Root Mean Squared Error:', round(np.sqrt(metrics.mean_squared_error(y_test, y_pred))/1000,2))

Mean Absolute Error: 991.1
Mean Squared Error: 9201689968.37
Root Mean Squared Error: 3033.43


MODELO DE REGRESION LINEAL - Logaritmo de Precio

In [90]:
x = datos_meli.drop('precio',axis=1)

#separte the predicting attribute into Y for model training 
y = datos_meli['precio']
y_ln = np.log(datos_meli['precio'])


X_train, X_test, y_train_ln, y_test_ln = train_test_split(
                                        x,
                                        y_ln,
                                        train_size   = 0.8,
                                        random_state = 1234,
                                        shuffle      = True
                                    )


print("Partición de entrenamiento")
print("-----------------------")
print(y_train_ln.describe())

print("Partición de test")
print("-----------------------")
print(y_test_ln.describe())

# Entrenamos el modelo
model_ln = LinearRegression()
model_ln.fit(X_train, y_train_ln)
y_pred_ln = model_ln.predict(X_test)

#  Saco exponencial de los valores de Y
y_test = np.exp(y_test_ln)
y_pred = np.exp(y_pred_ln)

# MIramos los resultados

print('Mean Absolute Error:', round(metrics.mean_absolute_error(y_test, y_pred)/1000,2))
print('Mean Squared Error:', round(metrics.mean_squared_error(y_test, y_pred)/1000,2))
print('Root Mean Squared Error:', round(np.sqrt(metrics.mean_squared_error(y_test, y_pred))/1000,2))


Partición de entrenamiento
-----------------------
count    19712.000000
mean        14.776689
std          0.506936
min         12.180755
25%         14.452087
50%         14.731801
75%         15.065412
max         18.064006
Name: precio, dtype: float64
Partición de test
-----------------------
count    4929.000000
mean       14.790185
std         0.519188
min        12.180755
25%        14.457364
50%        14.751604
75%        15.068274
max        18.526041
Name: precio, dtype: float64
Mean Absolute Error: 881.68
Mean Squared Error: 9166898967.02
Root Mean Squared Error: 3027.69
